In [5]:
# Various imports, setup
import pandas as pd
import numpy as np
import altair as alt

In [10]:
################
# Read in Data #
################
# For source see '../src/data_sources'

# USGS; 1980 to today, only 6.5+ quakes. 
usgs_quakes = pd.read_csv('../data/raw/usgs_1980_6_5_above.csv')

# NOAA; I think its 1980 to today - josh
noaa_quakes = pd.read_csv('../data/raw/noaa_quakes.tsv', sep='\t')





(1868, 22)

In [13]:
for source see ../src/data_sources
noaa_quakes.head()

,Search Parameters,Year,Mo,Dy,Hr,Mn,Sec,Tsu,Vol,Location Name,...,Total Missing,Total Missing Description,Total Injuries,Total Injuries Description,Total Damage ($Mil),Total Damage Description,Total Houses Destroyed,Total Houses Destroyed Description,Total Houses Damaged,Total Houses Damaged Description
0,"[""1980 <= Year >= 2021""]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1980.0,1.0,1.0,16.0,42.0,42.0,4259.0,NaN,"AZORES: TERCEIRA, ANGRA DO HEROISMO",...,NaN,NaN,600.0,3.0,5.0,2.0,NaN,NaN,NaN,NaN
2,NaN,1980.0,1.0,24.0,19.0,0.0,9.5,NaN,NaN,CALIFORNIA: LIVERMORE,...,NaN,NaN,50.0,1.0,11.5,3.0,NaN,NaN,NaN,NaN
3,NaN,1980.0,2.0,23.0,5.0,51.0,3.2,2123.0,NaN,RUSSIA: KURIL ISLANDS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1980.0,4.0,16.0,12.0,18.0,20.6,NaN,NaN,INDONESIA: JAVA: TASIKMALAJA,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
